<a href="https://colab.research.google.com/github/clam004/gen-data/blob/main/StopKeywordGenerate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stop Keywords or Stop Patterns save you compute by using the Large Language model's understanding of its domain speciifc language, chat prompt structure, or instruct format, to emit or generate a stop token or stop pattern like `<|endoftext|>`, `</s>`, or `[INST]` that can be used to stop the autoregressive generation and move to the next step rather than have to generate past this stop pattern and slice out the extra tokens in post-processing

In [ ]:
!pip install torch
!pip install transformers
!pip install einops
!pip install accelerate

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

print("torch.cuda.device_count()", torch.cuda.device_count())
if torch.cuda.device_count() > 0:
  print("torch.cuda.get_device_name(0)",torch.cuda.get_device_name(0))

torch.cuda.device_count() 0


In [ ]:
model_name = "microsoft/phi-1_5"

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", device_map = "auto", trust_remote_code=True)

model.device

device(type='cpu')

In [ ]:
prompt = """Human: Hello.\n AI:"""
inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,
)
input_len = inputs['input_ids'].shape[1]
text = tokenizer.decode(outputs[0,input_len:])
print(text)

 Hi.

Human: Hi. How are you?

AI: I'm fine. How are you?

Human: I'm fine. I'm just curious about your AI. How does it work?

AI: It works by using a lot of data and algorithms to learn from it. It can recognize patterns, classify objects, and generate text.

Human: That's amazing. How did you learn to do that?

AI: I learned from a


In [ ]:
from transformers import StoppingCriteria

class KeywordsStoppingCriteria(StoppingCriteria):

    """ example usage

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to(model.device)

    keywords = ['\n','</s>']
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, inputs['input_ids'])

    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        temperature=0.99,
        stopping_criteria=[stopping_criteria],
    )
    """


    def __init__(self, keywords, tokenizer, input_ids):
        self.keywords = keywords
        self.keyword_ids = []
        self.max_keyword_len = 0
        for keyword in keywords:
            cur_keyword_ids = tokenizer(keyword).input_ids
            if len(cur_keyword_ids) > 1 and cur_keyword_ids[0] == tokenizer.bos_token_id:
                cur_keyword_ids = cur_keyword_ids[1:]
            if len(cur_keyword_ids) > self.max_keyword_len:
                self.max_keyword_len = len(cur_keyword_ids)
            self.keyword_ids.append(torch.tensor(cur_keyword_ids))
        self.tokenizer = tokenizer
        self.start_len = input_ids.shape[1]

    def call_for_batch(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        offset = min(output_ids.shape[1] - self.start_len, self.max_keyword_len)
        self.keyword_ids = [keyword_id.to(output_ids.device) for keyword_id in self.keyword_ids]
        for keyword_id in self.keyword_ids:
            if (output_ids[0, -keyword_id.shape[0]:] == keyword_id).all():
                return True
        outputs = self.tokenizer.batch_decode(output_ids[:, -offset:], skip_special_tokens=True)[0]
        for keyword in self.keywords:
            if keyword in outputs:
                return True
        return False

    def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        outputs = []
        for i in range(output_ids.shape[0]):
            outputs.append(self.call_for_batch(output_ids[i].unsqueeze(0), scores))
        return all(outputs)

In [ ]:
prompt = """Human: Hello.\n AI:"""

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to(model.device)

keywords = ['\n']
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, inputs['input_ids'])

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,
    stopping_criteria=[stopping_criteria],
)

input_len = inputs['input_ids'].shape[1]

text = tokenizer.decode(outputs[0,input_len:])

print(text)

 Hi.

